In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import pytz
import datetime
from datetime import timezone
from numpy import trapz
import numpy as np
import os
import ast

In [ ]:
data_dir = '../../grafana_data/exp_multiple_workers_2/'
print(os.listdir(data_dir))

In [ ]:
filepath = data_dir + 'dejan-resnet50-256-ps-16-workers-16.csv'
gpu_info = pd.read_csv(filepath)
gpu_info['gpu-utilization'] = gpu_info['gpu-utilization'].astype(object)
gpu_info['gpu-memory'] = gpu_info['gpu-memory'].astype(object)
gpu_info['gpu-power-usage'] = gpu_info['gpu-power-usage'].astype(object)
gpu_info['gpu-temperature'] = gpu_info['gpu-temperature'].astype(object)

In [ ]:
grafana_gpu_power_usage = pd.read_csv(data_dir + 'grafana_gpu_power_usage.csv', sep=';')

In [ ]:
grafana_gpu_utilization = pd.read_csv(data_dir + 'grafana_gpu_utilization.csv', sep=';')

In [ ]:
grafana_gpu_memory = pd.read_csv(data_dir + 'grafana_gpu_memory.csv', sep=';')

In [ ]:
grafana_gpu_temperature = pd.read_csv(data_dir + 'grafana_gpu_temperature.csv', sep=';')

In [ ]:
def get_grafana_info(gpu_info, grafana_df, tolerance_min=15):
    grafana_df['Time'] = pd.to_datetime(grafana_df['Time'], format='%Y-%m-%dT%H:%M:%S')
    grafana_df['Time'] = grafana_df['Time'].dt.tz_convert(tz='Etc/GMT-2')

    start_time = pd.date_range(gpu_info['start-time'], periods=1) + pd.Timedelta(minutes=-tolerance_min)
    end_time = pd.date_range(gpu_info['end-time'], periods=1) + pd.Timedelta(minutes=tolerance_min)

    new_start = pd.date_range(start_time[0].replace(tzinfo=timezone.utc), periods=1)
    new_end = pd.date_range(end_time[0].replace(tzinfo=timezone.utc), periods=1)
    
    res_df = grafana_df.loc[(grafana_df['Series'] == gpu_info['nvidia-plugins'] + '-gpu0') & \
                            (grafana_df['Time'] > new_start[0]) & \
                            (grafana_df['Time'] <= new_end[0])]
    
    values = res_df['Value'].values
    
    return values

In [ ]:
def print_grafana_info(values, label='info'):
    if all(v == 0 for v in values):
        print('\tERROR')
    else:
        print('\t' + label + ' = ' + str(max(values)))

for index, row in gpu_info.iterrows():
    print(row['model'], row['batch-size'], row['nvidia-plugins'])
    tolerance_min = 10
    
    gpu_utilization = get_grafana_info(row, grafana_gpu_utilization, tolerance_min)
    print_grafana_info(gpu_utilization, 'gpu_utilization')
    gpu_info.at[index, 'gpu-utilization'] = gpu_utilization
    
    gpu_memory = get_grafana_info(row, grafana_gpu_memory)
    print_grafana_info(gpu_memory, 'gpu_memory')
    gpu_info.at[index, 'gpu-memory'] = gpu_memory
        
    gpu_power_usage = get_grafana_info(row, grafana_gpu_power_usage, tolerance_min)
    print_grafana_info(gpu_power_usage, 'gpu_power_usage')
    gpu_info.at[index, 'gpu-power-usage'] = gpu_power_usage
    
    gpu_temperature = get_grafana_info(row, grafana_gpu_temperature, tolerance_min)
    print_grafana_info(gpu_temperature, 'gpu_temperature')
    gpu_info.at[index, 'gpu-temperature'] = gpu_temperature
        
    print()

outpath = 'experiments-with-grafana.csv'
gpu_info.to_csv(outpath)

In [ ]:
gpu_info_full = pd.read_csv(outpath)

In [ ]:
def get_list(string, f=lambda x:float(x)):
    lst = str(string[1:-1]).split()
    return [f(x) for x in lst]

def get_max(model, column):
    model_df = gpu_info_full.loc[(gpu_info_full['model'] == model)].sort_values(by=['batch-size'])
    res = []
    x_axis = []
    for index, row in model_df.iterrows():
        res.append(max(get_list(model_df.at[index, column])))
        x_axis.append(model_df.at[index, 'batch-size'])
    
    return res, x_axis

titles = {'gpu-power-usage': 'Power', \
          'gpu-utilization': 'Utilization', \
          'gpu-memory': 'Memory', \
          'gpu-temperature': 'Temperature'}

units = {'gpu-power-usage': '[mW]', \
          'gpu-utilization': '[%]', \
          'gpu-memory': '[%]', \
          'gpu-temperature': '[C]'}

def scatter_all_max(column):
    alexnet_max_power, alexnet_xaxis = get_max('alexnet', column)
    resnet_max_power, resnset_xaxis = get_max('resnet50', column)
    inception_max_power, inception_xaxis = get_max('inception4', column)
    
    x_alexnet = np.arange(len(alexnet_max_power))
    x_resnet = np.arange(len(resnet_max_power))
    x_inception = np.arange(len(inception_max_power))
    
    plt.bar(x_alexnet - 0.2, alexnet_max_power, width=0.2)
    plt.bar(x_resnet, resnet_max_power, width=0.2)
    plt.bar(x_inception + 0.2, inception_max_power, width=0.2)
    plt.legend(['alexnet', 'resnet', 'inception'])
    plt.xticks(np.arange(len(alexnet_xaxis)), alexnet_xaxis)
    plt.xlabel('Batch Size')
    plt.title('GPU ' + titles[column] + ' ' + units[column] + ', Workers = 1, ' + 'PS = 1')
    
    #plt.savefig('plots/' + column + '-ps-16-workers-16.jpg', dpi=128)
    plt.show()
    
scatter_all_max('gpu-power-usage')
scatter_all_max('gpu-utilization')
scatter_all_max('gpu-memory')
scatter_all_max('gpu-temperature')